# 🛠️ Notebook: Einführung in Function Calling

In dem Notebook lernen wir, wie man LLMs dazu bringt, Funktionen aufzurufen und Tool-Interaktionen durchzuführen.  

## 📚 Quellen

- [OpenAI: Function Calling API](https://platform.openai.com/docs/guides/function-calling)
- [Ollama: Function Calling](https://ollama.com/blog/functions-as-tools)

---

Viel Erfolg beim Ausprobieren der Function Calling Features! 🤗

Beginnen wir damit ollama zu installieren. Anders als bei den vorherigen Notebooks, wo wir die OpenAI API genutzt haben, verwenden wir in diesem Notebook zur Abwechslung mal die Ollama API.


So wie `ollama`, bietet auch die Bibliothek `openai` die Möglichkeit, Requests an den Server mit den LLMs zu senden und unterstützt ebenfalls Function Calling.

[https://platform.openai.com/docs/guides/function-calling](https://platform.openai.com/docs/guides/function-calling)

In [11]:
%%capture
!uv add ollama

In [12]:
LLM_URL = "http://132.199.138.16:11434"
LLM_MODEL = "gpt-oss:20b" # Nicht jedes LLM unterstützt Function Calling. Gemma3 beispielsweise wurde nicht dafür trainiert. Das OpenSource LLM "gpt-oss:20b" von OpenAI hingegen schon.

In [13]:
from ollama import Client

# Verwenden wir zur Abwechslung mal die ollama Bibliothek statt openai. Tool Calling wird auch von openai unterstützt.
# Siehe hier: https://ollama.com/blog/tool-support
client = Client(
  host=LLM_URL
)

### 1. Beispiel: Tool Call mit Aktienkursen

Definieren wir zunächst eine Liste mit Tools. In unserem Beispiel nur **ein** Tool, das den aktuellen Aktienkurs für ein gegebenes Symbol zurückgibt. 
Die `openai`-Bibliothek erwartet jedes Tool in einem bestimmten JSON-Format. 

* `type`: OpenAI bietet prinzipiell auch andere Tool-Typen an (z.B. `web_search`), wir verwenden hier aber nur `function`.
* `name`: Der Name des Tools.
* `parameters`: Die Parameter, die die Funktion erwartet.
* `required`: Welche Parameter zwingend übergeben werden müssen.

In [14]:
tools = [{
    'type': 'function',
    'function': {
            'name': 'get_stock_price',
            'description': 'Get the current stock price for a company',
            'parameters': {
                'type': 'object',
                'properties': {
                    'symbol': {
                        'type': 'string',
                    },
                },
                'required': ['symbol'],
            },
    },
}]

In [15]:
# Starten wir mit einem Beispiel, in dem wir den aktuellen Aktienkurs von SAP abfragen.
response = client.chat(
    model=LLM_MODEL,
    messages=[{'role': 'user', 'content': 'Wie steht die aktuelle SAP Aktie?'}],
    tools=tools,
)

tool_calls = response['message']['tool_calls']
print(f"Tool calls: {tool_calls}")
# Argumente des Tool Calls und Funktion ausgeben
print(f"Function name: {tool_calls[0].function.name}")
print(f"Function arguments: {tool_calls[0].function.arguments}")

Tool calls: [ToolCall(function=Function(name='get_stock_price', arguments={'symbol': 'SAP'}))]
Function name: get_stock_price
Function arguments: {'symbol': 'SAP'}


In [16]:
# Fragen wir eine irrelevante Frage, die nichts mit Aktien zu tun hat.
# In dem Fall ist tool_calls None, da das LLM kein Tool aufrufen muss.
response = client.chat(
    model=LLM_MODEL,
    messages=[{'role': 'user', 'content': 'Was ist 1+1?'}],
    tools=tools,
)

# Schauen wir uns die Ausgabe an. Bei der Ausgabe sehen wir, dass tool_calls None ist.
print(response)

model='gpt-oss:20b' created_at='2025-11-21T17:49:39.929289Z' done=True done_reason='stop' total_duration=750125292 load_duration=148188708 prompt_eval_count=131 prompt_eval_duration=48113292 eval_count=54 eval_duration=539943919 message=Message(role='assistant', content='1\u202f+\u202f1\u202f=\u202f2.', thinking='The user asked in German: "Was ist 1+1?" which translates to "What is 1+1?" The answer is 2. Provide answer.', images=None, tool_name=None, tool_calls=None)


## Übungsaufgabe: Tool Execution - Funktionen tatsächlich ausführen

Wir implementieren jetzt ein Tool, das Informationen über Länder abruft. Bislang haben wir nur implementiert, dass das LLM ein Tool ggf. mit Parametern aufruft, aber die eigentliche Funktion dahinter wird nicht ausgeführt.

Gehen Sie für die Implementierung wie folgt vor:

1. Definieren Sie ein Tool `get_country_info` im JSON Schema mit Parameter `country`
2. Implementieren Sie die Funktion `get_country_info_from_api`, die Länder-Infos von der API abruft
3. Bauen Sie einen einfachen Workflow, der das LLM fragt und dann die Funktion aufruft


So funktioniert die API:

In [19]:
import requests

country = "Germany"  # Auch möglich: France, Italy, Japan, ...
url = f"https://restcountries.com/v3.1/name/{country}"
response = requests.get(url)
data = response.json()

# API gibt eine Liste zurück, wir nehmen das erste Element
country_data = data[0]
print(f"Name: {country_data['name']['common']}")
print(f"Hauptstadt: {country_data['capital'][0]}")
print(f"Bevölkerung: {country_data['population']}")
print(f"Region: {country_data['region']}")

Name: Germany
Hauptstadt: Berlin
Bevölkerung: 83491249
Region: Europe


<details>
<summary><b>Lösung anzeigen</b></summary>

```python
# 1. Tool Definition
tools = [{
    'type': 'function',
    'function': {
        'name': 'get_country_info',
        'description': 'Ruft Informationen über ein Land ab',
        'parameters': {
            'type': 'object',
            'properties': {
                'country': {
                    'type': 'string',
                    'description': 'Der Name des Landes (z.B. Germany, France, Japan)'
                }
            },
            'required': ['country']
        }
    }
}]

# 2. API-Funktion
def get_country_info_from_api(country):
    url = f"https://restcountries.com/v3.1/name/{country}"
    response = requests.get(url)
    data = response.json()[0]

    # Wir können die Informationen zu einem Text zusammenfassen:
    info = f"Land: {data['name']['common']}, Hauptstadt: {data['capital'][0]}, Bevölkerung: {data['population']}, Region: {data['region']}"
    return info


# 3. Workflow
messages = [{'role': 'user', 'content': 'Erzähl mir etwas über Norwegen.'}]

# LLM fragen
response = client.chat(model=LLM_MODEL,
                       messages=messages,
                       tools=tools,
                       options={"temperature": 0.0})

# Wurde ein Tool aufgerufen? Prüfen wir, ob der Key ['message']['tool_calls'] existiert
if response['message']['tool_calls']:
    country = response['message']['tool_calls'][0]['function']['arguments']['country']

    # Funktion ausführen
    country_info = get_country_info_from_api(country)

    # Prompt erstellen für finale Antwort
    prompt = f"Hier sind die Informationen über {country}:\n{country_info}.\nBitte fasse diese Informationen in zwei Sätzen kurz zusammen."

    # Zurück ans LLM
    messages = [{'role': 'user', 'content': prompt}]
    final_response = client.chat(
        model=LLM_MODEL, messages=messages, options={"temperature": 0.0})

    print(f"\nFinale Antwort: {final_response['message']['content']}")
```

</details>